## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import gc

pd.options.mode.chained_assignment = None

In [2]:
preprocess_path = '../data/mimiciii/preprocess_data/'
temporal_path = '../data/mimiciii/temporal_dataset/'

## Load Data

In [4]:
datas = list()
paths = sorted(os.listdir(preprocess_path))
for path in tqdm(paths):
    datas.append(pd.read_csv(preprocess_path + path))

  0%|                                                                                                                                                                               | 0/28 [00:00<?, ?it/s]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 21%|███████████████████████████████████▊                                                                                                                                   | 6/28 [00:17<01:07,  3.06s/it]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 29%|███████████████████████████████████████████████▋                                                    

# Look Data characteristics

In [6]:
for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [03:21<00:00,  7.21s/it]


In [7]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


In [12]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
DIAGNOSES_ICD.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,140,21,111970,11.0,99592
1,307,38,185910,3.0,99592
2,477,61,189535,8.0,99591
3,490,62,116009,4.0,99591
4,505,64,172056,3.0,99591


In [14]:
i = paths.index('ICUSTAYS.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
1,372,275,129886,219649,carevue,CCU,CCU,7,7,2170-10-07 11:28:53,2170-10-14 14:38:07,7.1314
2,410,305,194340,217232,metavision,SICU,SICU,33,33,2129-09-03 12:31:31,2129-09-05 23:00:50,2.4370
3,432,323,143334,264375,carevue,MICU,MICU,23,23,2120-01-11 15:48:28,2120-01-14 16:24:43,3.0252
4,439,329,172132,217001,carevue,MICU,MICU,50,50,2154-04-05 16:36:51,2154-04-08 18:14:48,3.0680


In [15]:
i = paths.index('PATIENTS.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,235,250,F,2164-12-27,2188-11-22,2188-11-22,NaT,1
1,252,269,M,2130-09-30,NaT,NaT,NaT,0
2,646,685,F,2126-07-29,NaT,NaT,NaT,0
3,650,689,F,2128-12-27,NaT,NaT,NaT,0
4,651,690,M,2109-09-24,2189-07-30,2189-07-30,2189-07-30,1


In [16]:
i = paths.index('ADMISSIONS.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaT,NaT,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
1,457,357,145674,2198-08-02 04:49:00,2198-10-26 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-08-02 02:03:00,2198-08-02 05:37:00,SEIZURE,0,1
2,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-11-01 18:01:00,2198-11-01 23:06:00,SEPSIS,0,1
3,471,366,134462,2164-11-18 20:27:00,2164-11-22 15:18:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,SINGLE,HISPANIC OR LATINO,2164-11-18 10:52:00,2164-11-18 21:31:00,SEPSIS,0,1
4,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Private,NaN,CATHOLIC,MARRIED,WHITE,NaT,NaT,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,1,1


In [22]:
i = paths.index('DRGCODES.csv')
DRGCODES = datas[i].copy()
DRGCODES.head()

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,2861,9102,154215,HCFA,385,"NEONATES, DIED OR TRANSFERRED TO ANOTHER ACUTE...",NaN,NaN
1,2933,20617,178293,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN
2,3019,14831,160882,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN
3,3027,15204,168396,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN
4,4357,5424,157250,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN


# Create dataset

In [26]:
dataset = DIAGNOSES_ICD[['SUBJECT_ID', 'HADM_ID']].copy()
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset

,SUBJECT_ID,HADM_ID
0,21,111970
1,38,185910
2,61,189535
3,62,116009
4,64,172056
...,...,...
5404,99836,101018
5405,99865,189346
5406,99912,189380
5407,99985,176670


## Add gender and age to dataset

In [27]:
dataset['DOB'] = np.nan
dataset['Gender'] = np.nan

for i in PATIENTS.index:
    subject_id = PATIENTS['SUBJECT_ID'].loc[i]
    gender = PATIENTS['GENDER'].loc[i]
    dob = PATIENTS['DOB'].loc[i]
    index = dataset[dataset['SUBJECT_ID'] == subject_id].index
    dataset['DOB'].loc[index] = dob
    dataset['Gender'].loc[index] = gender
    
dataset['Gender'] = dataset['Gender'].apply(lambda x : 1 if x == 'M' else 0)
dataset

,SUBJECT_ID,HADM_ID,DOB,Gender
0,21,111970,2047-04-04 00:00:00,1
1,38,185910,2090-08-31 00:00:00,1
2,61,189535,2063-10-21 00:00:00,1
3,62,116009,2044-05-08 00:00:00,1
4,64,172056,2116-06-27 00:00:00,0
...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0
5405,99865,189346,2107-06-25 00:00:00,0
5406,99912,189380,2068-09-26 00:00:00,1
5407,99985,176670,2127-04-08 00:00:00,1


## Add death time, admit time, discharge time to dataset

In [28]:
dataset['DEATHTIME'] = np.nan
dataset['ADMITTIME'] = np.nan
dataset['DISCHTIME'] = np.nan

for i in ADMISSIONS.index:
    subject_id = ADMISSIONS['SUBJECT_ID'].loc[i]
    hadm_id = ADMISSIONS['HADM_ID'].loc[i]
    death_time = ADMISSIONS['DEATHTIME'].loc[i]
    admittime = ADMISSIONS['ADMITTIME'].loc[i]
    dischtime = ADMISSIONS['DISCHTIME'].loc[i]
    index = dataset.query(f'SUBJECT_ID == {subject_id} & HADM_ID == {hadm_id}').index
    dataset['DEATHTIME'].loc[index] = death_time
    dataset['ADMITTIME'].loc[index] = admittime
    dataset['DISCHTIME'].loc[index] = dischtime
    
dataset

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME,ADMITTIME,DISCHTIME
0,21,111970,2047-04-04 00:00:00,1,2135-02-08 02:08:00,2135-01-30 20:50:00,2135-02-08 02:08:00
1,38,185910,2090-08-31 00:00:00,1,NaT,2166-08-10 00:28:00,2166-09-04 11:30:00
2,61,189535,2063-10-21 00:00:00,1,2119-02-03 01:35:00,2119-01-04 18:12:00,2119-02-03 01:35:00
3,62,116009,2044-05-08 00:00:00,1,NaT,2113-02-15 00:19:00,2113-02-19 15:30:00
4,64,172056,2116-06-27 00:00:00,0,NaT,2143-03-03 09:25:00,2143-03-18 13:01:00
...,...,...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0,NaT,2116-05-21 19:23:00,2116-07-17 16:30:00
5405,99865,189346,2107-06-25 00:00:00,0,2184-12-30 00:01:00,2184-12-10 10:30:00,2184-12-30 00:01:00
5406,99912,189380,2068-09-26 00:00:00,1,NaT,2153-04-13 16:05:00,2153-04-27 13:40:00
5407,99985,176670,2127-04-08 00:00:00,1,NaT,2181-01-27 02:47:00,2181-02-12 17:05:00


## change time by ICUSTAY_ID in dataset

In [29]:
for i in ICUSTAYS.index:
    subject_id = ICUSTAYS['SUBJECT_ID'].loc[i]
    hadm_id = ICUSTAYS['HADM_ID'].loc[i]
    intime = ICUSTAYS['INTIME'].loc[i]
    outtime = ICUSTAYS['OUTTIME'].loc[i]
    index = dataset.query(f'SUBJECT_ID == {subject_id} & HADM_ID == {hadm_id}').index
    if dataset['ADMITTIME'].isnull()[index[0]]:
        dataset['ADMITTIME'].loc[index[0]] = intime
    elif dataset['ADMITTIME'].loc[index[0]] - intime > pd.Timedelta('00:00:00'):
        dataset['ADMITTIME'].loc[index[0]] = intime
    if dataset['DISCHTIME'].isnull()[index[0]]:
        dataset['DISCHTIME'].loc[index[0]] = outtime
    elif dataset['DISCHTIME'].loc[index[0]] - outtime < pd.Timedelta('00:00:00'):
        dataset['DISCHTIME'].loc[index[0]] = outtime


dataset.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME,ADMITTIME,DISCHTIME
0,21,111970,2047-04-04 00:00:00,1,2135-02-08 02:08:00,2135-01-30 20:50:00,2135-02-08 05:38:46
1,38,185910,2090-08-31 00:00:00,1,NaT,2166-08-10 00:28:00,2166-09-04 13:39:23
2,61,189535,2063-10-21 00:00:00,1,2119-02-03 01:35:00,2119-01-04 18:12:00,2119-02-03 01:35:00
3,62,116009,2044-05-08 00:00:00,1,NaT,2113-02-15 00:19:00,2113-02-19 15:30:00
4,64,172056,2116-06-27 00:00:00,0,NaT,2143-03-03 09:25:00,2143-03-18 13:01:00
...,...,...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0,NaT,2116-05-21 19:23:00,2116-07-17 16:30:00
5405,99865,189346,2107-06-25 00:00:00,0,2184-12-30 00:01:00,2184-12-10 10:30:00,2184-12-31 02:56:46
5406,99912,189380,2068-09-26 00:00:00,1,NaT,2153-04-13 16:05:00,2153-04-27 13:40:00
5407,99985,176670,2127-04-08 00:00:00,1,NaT,2181-01-27 02:47:00,2181-02-12 17:05:00


## Add re-admission to dataset

In [30]:
dataset['re_admission'] = np.nan

dataset['re_admission'].loc[0] = 0
for i in dataset.index[1:]:
    if dataset['SUBJECT_ID'].loc[i] == dataset['SUBJECT_ID'].loc[i - 1]:
        dataset['re_admission'].loc[i] = 1
    else:
        dataset['re_admission'].loc[i] = 0

dataset

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME,ADMITTIME,DISCHTIME,re_admission
0,21,111970,2047-04-04 00:00:00,1,2135-02-08 02:08:00,2135-01-30 20:50:00,2135-02-08 05:38:46,0.0
1,38,185910,2090-08-31 00:00:00,1,NaT,2166-08-10 00:28:00,2166-09-04 13:39:23,0.0
2,61,189535,2063-10-21 00:00:00,1,2119-02-03 01:35:00,2119-01-04 18:12:00,2119-02-03 01:35:00,0.0
3,62,116009,2044-05-08 00:00:00,1,NaT,2113-02-15 00:19:00,2113-02-19 15:30:00,0.0
4,64,172056,2116-06-27 00:00:00,0,NaT,2143-03-03 09:25:00,2143-03-18 13:01:00,0.0
...,...,...,...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0,NaT,2116-05-21 19:23:00,2116-07-17 16:30:00,0.0
5405,99865,189346,2107-06-25 00:00:00,0,2184-12-30 00:01:00,2184-12-10 10:30:00,2184-12-31 02:56:46,0.0
5406,99912,189380,2068-09-26 00:00:00,1,NaT,2153-04-13 16:05:00,2153-04-27 13:40:00,0.0
5407,99985,176670,2127-04-08 00:00:00,1,NaT,2181-01-27 02:47:00,2181-02-12 17:05:00,0.0


# Split time

In [31]:
dataset['Age'] = np.nan
for hour in range(1, 0, -1):
    hour_period = f'0{hour}:00:00'
    index = 0
    split = pd.DataFrame(columns = dataset.columns)
    for i in tqdm(dataset.index):
        intime, outtime = dataset['ADMITTIME'].loc[i], dataset['DISCHTIME'].loc[i]
        if dataset['ADMITTIME'].isnull()[i] or dataset['ADMITTIME'].isnull()[i]:
            continue
        while outtime - intime > pd.Timedelta(hour_period):
            #['SUBJECT_ID', 'HADM_ID', 'DOB', 'Gender', 'DEATHTIME', 'ADMITTIME', 'DISCHTIME', 're_admission', 'Age']
            split.loc[index] = [
                dataset['SUBJECT_ID'].loc[i],
                dataset['HADM_ID'].loc[i],
                dataset['DOB'].loc[i],
                dataset['Gender'].loc[i],
                dataset['DEATHTIME'].loc[i],
                intime,
                intime + pd.Timedelta(hour_period),
                dataset['re_admission'].loc[i],
                (intime.to_pydatetime() - dataset['DOB'].loc[i].to_pydatetime()).days / 365
            ]
            intime += pd.Timedelta(hour_period)
            index += 1

        split.loc[index] = [
            dataset['SUBJECT_ID'].loc[i],
            dataset['HADM_ID'].loc[i],
            dataset['DOB'].loc[i],
            dataset['Gender'].loc[i],
            dataset['DEATHTIME'].loc[i],
            intime,
            outtime,
            dataset['re_admission'].loc[i],
            (intime.to_pydatetime() - dataset['DOB'].loc[i].to_pydatetime()).days / 365
        ]

    split.drop('DOB', axis=1, inplace=True)
    split = split[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']]
    split.columns = ['SUBJECT_ID', 'HADM_ID', 'STARTTIME', 'ENDTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']
    split.to_csv(temporal_path + f'dataset_split_{hour}_hour.csv', index=False)

  0%|▏                                                                                                                                                                  | 6/5409 [00:05<1:19:52,  1.13it/s]


KeyboardInterrupt: 